# Tracy Reinhold: Global Security and Resilience Authority

### "You can get ahead of threats by being informed and having the technological capabilities to actually take action on that intelligence once you’ve acquired it. So if you’re consistently in a reactive mode, you will eventually fail. By proactively reducing risk and vulnerability, we can create an environment that allows a company to be successful." —via [Security Magazine](https://www.securitymagazine.com/articles/96603-5-minutes-with-tracy-reinhold-critical-event-management-for-enterprise-resilience)

### We consider the arc of a critical event and the way Tracy considers the future of predictive approaches with AI.

| Before the 💥 Boom | During the 💥 Boom | After the 💥 Boom |
|---|---|---|

Note that there are three phases to a critical event: before, during, and after. The critical event is often referred to as "💥 boom" or "💥 bang."

Instructions for getting cozy with this AI recipe are on the [GitHub page's README](https://aka.ms/CAIK-repo). 

# 🧑‍🍳 Recipe for AI-driven Operational Resilience

- ~500 tokens from COMPLETION Pre-trained Foundation Model
- ~1000 tokens from EMBEDDING Pre-trained Foundation Model
 
First, register a variety of pieces of data into the memory store with the EMBEDDING model's help.

Then we look for any patterns that could be troubling.

Take all those patterns and generate a quick report.

> [!IMPORTANT]
> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

# Gather the core tools 🧰

When running the following cell, if asked to "select your kernel" (note this will be referring to the Jupyter notebook's kernel and not Semantic Kernel) then choose `.NET Interactive` from the available menu options.

In [ ]:
// Load some helper functions, e.g. to load values from settings.json
#!import ../config/Settings.cs 

#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta3"
#r "nuget: System.Linq.Async, 6.0.1"

# Fire up a kernel with TWO models 🔥

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

var memoryBuilder = new MemoryBuilder();
var kernelBuilder = new KernelBuilder();

// gpt-3.5-turbo is used by default
model = "gpt-4";

Console.WriteLine($"Using 🧱 Model: {model}");

if (useAzureOpenAI)
{
    memoryBuilder.WithAzureTextEmbeddingGenerationService("text-embedding-ada-002", azureEndpoint, apiKey);
    kernelBuilder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
}
else
{
    memoryBuilder.WithOpenAITextEmbeddingGenerationService("text-embedding-ada-002", apiKey);
    kernelBuilder.WithOpenAIChatCompletionService(model, apiKey, orgId);
}

memoryBuilder.WithMemoryStore(new VolatileMemoryStore());

var kernel = kernelBuilder.Build();
var memory = memoryBuilder.Build();

# Generate embeddings for later 🧲 gathering

In [ ]:
const string MemoryCollectionName = "events";

string[] reports = { 
    "7:00am GMT: all stores report normal activity.",
    "7:30am GMT: an email arrived in VP <name redacted> inbox that was threatening and abusive.",
    "8:30am GMT: retail operations in London report normal opening activity.",
    "8:00am GMT: customer support loads are normal.",
    "9:00am GMT: a noisy protest is reported outside the London store.",
    "9:30am GMT: retail operations in New York report a theft has occurred.",
    "10:00am GMT: retail operations in Los Angeles report normal closing activity.",
    "10:30am GMT: retail operations in Las Vegas reports flooding has shut down the store.",
    "11:00am GMT: retail operations in London report a theft has occured.",
    "11:30am GMT: security detail for VP <name redacted> reports a suspicious package has been found in the mail room."
};

for (int i = 0; i < reports.Length; i++)
{
    await memory.SaveInformationAsync(MemoryCollectionName, id: $"ev{i}", text: reports[i]);
}


# Experiment 🧪 with what you think is relevant and important 

Let's just find a few needles a potential haystack.

In [ ]:
var questions = new[]
{
    "something bad",
    "all things okay",
};

foreach (var q in questions)
{
    var response = await memory.SearchAsync(MemoryCollectionName, q).FirstOrDefaultAsync();
    Console.WriteLine($"{q}\n> {response?.Metadata.Text}");
}

Let's instead try to cast a wider net for a query.

In [ ]:
using Microsoft.SemanticKernel.Memory;

var query = "all the bad things";
var myResults = "";

var memoryResults = memory.SearchAsync(MemoryCollectionName, query, limit: 5, minRelevanceScore: 0.5);

int i = 0;
await foreach (MemoryQueryResult memoryResult in memoryResults)
{
    Console.WriteLine($"Result {++i}:");
    Console.WriteLine("  Text: " + memoryResult.Metadata.Text);
    Console.WriteLine("  Relevance: " + memoryResult.Relevance);

    if (memoryResult.Relevance > 0.7) {
        myResults += memoryResult.Metadata.Text + "\n";
    }
    
    Console.WriteLine();
}

In [ ]:
Console.WriteLine(myResults);

## 🔌 We'll use a few readily available Plugins

In [ ]:
TimeSpan startTime = TimeSpan.FromTicks(DateTime.Now.Ticks);

// Load Design Thinking Plugin
var pluginsSKDirectory = "plugins-sk";
var secFunctions = kernel.ImportSemanticFunctionsFromDirectory(pluginsSKDirectory, "SecurityKit");

// Run the Function EmpathizeTable
//var result = await kernel.RunAsync( myResults, secFunctions["SuggestNextSteps"] );
var result = await kernel.RunAsync( myResults, secFunctions["SitRepPlain"] );

TimeSpan endTime = TimeSpan.FromTicks(DateTime.Now.Ticks);
TimeSpan elapsed = endTime - startTime;

// Return the result to the Notebook
var resultString = result.GetValue<string>();

Console.WriteLine(resultString);

# Plate the partially finished meal 🍽️

In [ ]:
using Microsoft.AspNetCore.Html;
using Markdig;

var myPipeline = new MarkdownPipelineBuilder().UseAdvancedExtensions().Build();
var myResult2HTML = Markdown.ToHtml(resultString, myPipeline);
var myHTMLContent = new HtmlContentBuilder();

myHTMLContent.AppendHtml(myResult2HTML);
myHTMLContent

# Let's look at the science 📐

In [ ]:
using System.Text.Json;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI;

var usageresult = result.FunctionResults.SelectMany(l => l.GetModelResults() ?? Enumerable.Empty<ModelResult>()).LastOrDefault()?.GetOpenAIChatResult()?.Usage;
var injson = JsonSerializer.Serialize(usageresult, new JsonSerializerOptions { WriteIndented = true });
Console.WriteLine($"🧱 The MODEL used:\n\n> {model}\n");
Console.WriteLine($"💸 How many TOKENS were used?\n\n{injson}\n");
String functionName = "SitRepPlain";

var reqjson = JsonSerializer.Serialize(secFunctions[functionName].RequestSettings, new JsonSerializerOptions { WriteIndented = true });
var doc = JsonDocument.Parse(reqjson);
double temperatureValue = doc.RootElement.GetProperty("temperature").GetDouble();
double topPValue = doc.RootElement.GetProperty("top_p").GetDouble();

Console.WriteLine($"🌡️ What TEMPERATURE setting was used?\n\n> {temperatureValue} (0 is 🥶 ; 1 is 🥵)\n");
Console.WriteLine($"🙊 What TOP P setting was used?\n\n> {topPValue} (0 is 😑 ; 1 is 🤯)");